# 1. 데이터베이스에서 데이터 로드

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta, time 
import json

with open("./db_config.json", "r") as f:
    config = json.load(f)
    
DB_USER = config["DB_USER"]
DB_PASSWORD = config["DB_PASSWORD"]
DB_HOST = config["DB_HOST"]
DB_NAME = config["DB_NAME"]
DB_PORT = config["DB_PORT"]

# SQLAlchemy 엔진 생성
engine_url = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(engine_url)
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
except Exception as e:
    print(f"데이터베이스 연결 오류: {e}")
    exit()

# --- 데이터베이스에서 데이터 로드하는 함수 ---
def load_table_to_df(table_name, engine):
    """지정된 테이블에서 모든 데이터를 Pandas DataFrame으로 로드합니다."""
    try:
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, engine)
        print(f"'{table_name}' 테이블 로드 완료. {len(df)} 행.")
        # 날짜/시간 컬럼 타입 변환 (필요시)
        if 'game_date' in df.columns:
            df['game_date'] = pd.to_datetime(df['game_date'])
        if 'accident_date' in df.columns:
            df['accident_date'] = pd.to_datetime(df['accident_date'])
        if 'weather_date' in df.columns:
            df['weather_date'] = pd.to_datetime(df['weather_date'])
        # start_time, end_time, weather_time은 문자열로 로드될 수 있으므로, 필요시 time 객체로 변환
        return df
    except Exception as e:
        print(f"'{table_name}' 테이블 로드 중 오류 발생: {e}")
        return pd.DataFrame() # 오류 발생 시 빈 DataFrame 반환

# --- 데이터 로드 ---
stadium_df = load_table_to_df('stadium', engine)
sports_game_df = load_table_to_df('sports_game', engine)
traffic_accident_df = load_table_to_df('traffic_accident', engine)
weather_df = load_table_to_df('weather', engine)

# 데이터베이스 연결 종료
if 'engine' in locals() and engine:
    engine.dispose()
    print("\n데이터베이스 연결이 종료되었습니다.")

MySQL 데이터베이스에 성공적으로 연결되었습니다.
'stadium' 테이블 로드 완료. 60 행.
'sports_game' 테이블 로드 완료. 3631 행.
'traffic_accident' 테이블 로드 완료. 34032 행.
'weather' 테이블 로드 완료. 450811 행.

데이터베이스 연결이 종료되었습니다.


# 2. 데이터셋을 구성할 지역구 입력
* 사용자로부터 지역명을 입력받아 데이터 셋 구성
## 사용자로부터 입력 받을 요소
* 지역 : 지역별에 입력을 안하거나 전국을 입력하면 모든 지역 데이터를 가져옴. 
    * 송파 : 특정 지역구 입력 (여러 지역구 입력은 , 를 통해서 구분 (송파,마포))
    * 전체 : 전체 라고 입력하거나 아무것도 입력 안할 시에 전국 데이터를 가져옴
* 종목 : 데이터셋으로 포함할 종목입력.
    * 야구 : 특정 종목 입력 (여러 종목 입력은 , 를 통해서 구분 (야구,축구))
    * 전체 : 전체 라고 입력하거나 아무것도 입력 안할 시에 전체 종목 데이터를 가져옴
* 요일 : 요일별로 데이터셋 구성.
    * 월요일 : 특정 요일 입력 (여러 요일 입력은 , 를 통해서 구분 (월,수,금))
    * 전체 : 전체 라고 입력하거나 아무것도 입력 안할 시에 전체 요일 데이터를 가져옴
* 날짜범위 : 특정한 날짜 범위로 데이터 셋 구성.
    * 20230401,20231131 : 특정 날짜 범위를 시작일,종료일 형식으로 입력
    * 전체 : 전체 라고 입력하거나 아무것도 입력 안할 시에 2023년, 2024년 전체 데이터를 가져옴

In [14]:
START_DATE = pd.to_datetime("20230101") # 분석 시작일
END_DATE = pd.to_datetime("20241231")   # 분석 종료일
stadium_region_list = stadium_df['region'].unique().tolist()
sports_list = ['야구','축구','배구','농구']
weekday_list = ['월','화','수','목','금','토','일']
TARGET_REGION = []
TARGET_SPORTS = []
TARGET_WEEKDAYS = []
TARGET_DATES = []
while True:
    print("========== 지역구 입력 ===========================================================")
    print("= 데이터 셋을 구성할 지역구를 입력하세요.                                            =")
    print("= 하나의 구만 적거나 여러 지역구를 , 로 구분해서 입력하세요. (예시:송파구,마포구)         =")
    print("= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 전국의 데이터를 가져옵니다. =")
    print("=================================================================================")
    input_region = input("데이터셋을 구성할 지역구를 입력해주세요 (예시:송파구,마포구) :")
    if len(input_region.strip()) == 0 or input_region.strip() == '전체' :
        TARGET_REGION = stadium_region_list
        break
    elif len(input_region.split(',')) > 1 :
        input_region = input_region.strip().split(',')
    else :
        input_region = [input_region]
    for iv in input_region:
        for item in stadium_region_list:
            item_split = item.split(' ')[1]
            if iv in item_split:
                TARGET_REGION.append(item)
    if len(TARGET_REGION) > 0:
        break
    else:
        print(f"입력하신 {input_region}은 정보가 없습니다. 다시 입력해주세요.")
while True:
    print("========== 스포츠 종목 입력 =======================================================")
    print("= 데이터 셋을 구성할 스포츠 종목을 입력하세요.                                        =")
    print("= 하나의 스포츠만 적거나 여러 스포츠를 , 로 구분해서 입력하세요. (예시:야구,배구)         =")
    print("= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 모든 데이터를 가져옵니다.   =")
    print("=================================================================================")
    input_sports = input("데이터셋을 구성할 스포츠를 입력해주세요 (예시:야구,배구) :")
    if len(input_sports.strip()) == 0 or input_sports.strip() == '전체' :
        TARGET_SPORTS = sports_list
        break
    elif len(input_sports.split(',')) > 1 :
        input_sports = input_sports.strip().split(',')
    else :
        input_sports = [input_sports]
    for iv in input_sports:
        for item in sports_list:
            if iv in item:
                TARGET_SPORTS.append(item)
    if len(TARGET_SPORTS) > 0:
        break
    else:
        print(f"입력하신 {input_sports}은 정보가 없습니다. 다시 입력해주세요.")
while True:
    print("========== 요일 입력 =============================================================")
    print("= 데이터 셋을 구성할 요일을 입력하세요.                                              =")
    print("= 하나의 요일만 적거나 여러 요일을 , 로 구분해서 입력하세요. (예시:월,화,수)             =")
    print("= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 모든 데이터를 가져옵니다.   =")
    print("=================================================================================")
    input_weekday = input("데이터셋을 구성할 요일을 입력해주세요 (예시:월,화,수) :")
    if len(input_weekday.strip()) == 0 or input_weekday.strip() == '전체' :
        TARGET_WEEKDAYS = weekday_list
        break
    elif len(input_weekday.split(',')) > 1 :
        input_weekday = input_weekday.strip().split(',')
    else :
        input_weekday = [input_weekday]
    for iv in input_weekday:
        for item in weekday_list:
            if iv in item:
                TARGET_WEEKDAYS.append(item)
    if len(TARGET_WEEKDAYS) > 0:
        break
    else:
        print(f"입력하신 {input_weekday}은 정보가 없습니다. 다시 입력해주세요.")
while True:
    print("========== 날짜 범위 입력 =========================================================")
    print("= 데이터 셋을 구성할 날짜 범위를 입력하세요.                                          =")
    print("= 날짜 범위를 시작일,종료일 형식으로 입력하세요. (예시:20230101,20241231)              =")
    print("= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 모든 데이터를 가져옵니다.   =")
    print("=================================================================================")
    input_date = input("데이터셋을 구성할 날짜 범위를 입력해주세요 (예시:20230101,20241231) :")
    if len(input_date.strip()) == 0 or input_date.strip() == '전체' :
        TARGET_DATES = [START_DATE, END_DATE]
        break
    elif len(input_date.split(',')) == 2 :
        input_parts = input_date.strip().split(',')

        user_start_date = pd.to_datetime(input_parts[0].strip()) # 각 부분도 strip()
        user_end_date = pd.to_datetime(input_parts[1].strip())   # 각 부분도 strip()
        # 1. 사용자가 입력한 시작일이 종료일보다 이전이거나 같은지 확인
        if user_start_date > user_end_date:
            print(f"입력 오류: 시작일({user_start_date.strftime('%Y%m%d')})이 종료일({user_end_date.strftime('%Y%m%d')})보다 늦습니다. 다시 입력해주세요.")
        # 2. 사용자가 입력한 날짜가 전체 범위 내에 있는지 확인
        elif user_start_date >= START_DATE and user_end_date <= END_DATE:
            TARGET_DATES = [user_start_date, user_end_date]
            break
        else:
            print(f"입력하신 날짜 범위가 유효하지 않습니다. 전체 기간은 {START_DATE.strftime('%Y%m%d')}부터 {END_DATE.strftime('%Y%m%d')}까지입니다. 다시 입력해주세요.")
    else :
        print(f"입력하신 '{input_date}'은(는) 형식이 올바르지 않습니다. 'YYYYMMDD,YYYYMMDD' 또는 '전체'를 입력해주세요.")
    if len(TARGET_DATES) == 2:
        break

print(f"선택한 지역: {TARGET_REGION}")
print(f"선택한 스포츠: {TARGET_SPORTS}")
print(f"선택한 요일: {TARGET_WEEKDAYS}")
print(f"선택한 날짜: {TARGET_DATES}")

========== 지역구 입력 ===========================================================
= 데이터 셋을 구성할 지역구를 입력하세요.                                            =
= 하나의 구만 적거나 여러 지역구를 , 로 구분해서 입력하세요. (예시:송파구,마포구)         =
= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 전국의 데이터를 가져옵니다. =
========== 스포츠 종목 입력 =======================================================
= 데이터 셋을 구성할 스포츠 종목을 입력하세요.                                        =
= 하나의 스포츠만 적거나 여러 스포츠를 , 로 구분해서 입력하세요. (예시:야구,배구)         =
= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 모든 데이터를 가져옵니다.   =
========== 요일 입력 =============================================================
= 데이터 셋을 구성할 요일을 입력하세요.                                              =
= 하나의 요일만 적거나 여러 요일을 , 로 구분해서 입력하세요. (예시:월,화,수)             =
= 아무것도 입력하지 않고 enter키를 누르거나 전체 라고 입력하면 모든 데이터를 가져옵니다.   =
========== 날짜 범위 입력 =========================================================
= 데이터 셋을 구성할 날짜 범위를 입력하세요.                                          =
= 날짜 범위를 시작일,종료일 형식으로 입력하세요. (예시:20230101,20241231)  

# 3. 입력한 설정을 기반으로 데이터 셋 구성

In [ ]:
# 설정한 지역 데이터 추가와 설정한 범위의 날짜 추가
base_df = pd.DataFrame()
base_df['region'] = pd.NA
base_df['date'] = pd.NA
date_range = pd.date_range(start=TARGET_DATES[0], end=TARGET_DATES[1], freq='D')
for region in TARGET_REGION:
    temp_df = pd.DataFrame({'date': date_range})
    temp_df['region'] = region 
    base_df = pd.merge(base_df,temp_df,how='outer')

In [28]:
base_df

,region,date
0,강원 강릉시,2023-01-01
1,강원 강릉시,2023-01-02
2,강원 강릉시,2023-01-03
3,강원 강릉시,2023-01-04
4,강원 강릉시,2023-01-05
...,...,...
28504,충북 청주시,2024-12-27
28505,충북 청주시,2024-12-28
28506,충북 청주시,2024-12-29
28507,충북 청주시,2024-12-30


In [31]:
# 스타디움 정보 가져오기
# region, sports_type, stadium_code
base_df['sports_type'] = pd.NA
base_df['stadium_code'] = pd.NA
if not stadium_df.empty:
    for region in TARGET_REGION:
        temp_df = pd.DataFrame({'date': date_range})
        

In [ ]:


# TARGET_REGION에 따른 데이터 셋 구성
# 데이터셋으로 더 구성할 컬럼 : 관중수, 경기 시작 시간, 


# 스포츠경기 정보 가져오기
if not sports_game_df.empty and stadium_codes_in_region:
    games_in_region_df = sports_game_df[sports_game_df['stadium_code'].isin(stadium_codes_in_region)]
    games_in_region_df = games_in_region_df.rename(columns={'game_date': 'date'})
    #print(games_in_region_df)
    if not games_in_region_df.empty:
        games_in_region_df["match_type"] = (
            games_in_region_df["match_type"]
                .replace({"페넌트레이스": "정규시즌",
                        "순위결정전": "정규시즌",
                        "순위결정정": "정규시즌",   # 오타까지 함께 처리
                        '조별리그' : "정규시즌",
                        "0": "정규시즌"})
                # ➋ 라운드 표기(1R ~ 33R 등) → 정규시즌
                .str.replace(r"^\d+R$", "정규시즌", regex=True)
        )
        games_in_region_df["match_type"] = (
            games_in_region_df["match_type"]
                .replace({'와일드카드':"포스트시즌",
                        '준플레이오프':"포스트시즌", 
                        '플레이오프':"포스트시즌", 
                        '한국시리즈':"포스트시즌",
                        '파이널 라운드A':"포스트시즌",
                        '파이널 라운드B':"포스트시즌",
                        '챔피언결정전':"포스트시즌", 
                        '준결승':"포스트시즌", 
                        '결승':"포스트시즌"})
        )
        game_summary_df = games_in_region_df.groupby('date').agg(
            game_count=('stadium_code', 'count'),
            sports_types_list=('sports_type', lambda x: list(set(x))),
            is_post_season_list=('match_type', lambda x: 1 if any('포스트시즌' in str(mt).lower() for mt in x) else 0)
        ).reset_index()
        
        game_summary_df['sports_types'] = game_summary_df['sports_types_list'].apply(lambda x: ','.join(sorted(x)) if x else '없음')
        game_summary_df['is_post_season'] = game_summary_df['is_post_season_list']
        game_summary_df = game_summary_df[['date', 'game_count', 'sports_types', 'is_post_season']]
        base_df = pd.merge(base_df, game_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 경기 정보 없음.")
        base_df['game_count'] = 0
        base_df['sports_types'] = '없음'
        base_df['is_post_season'] = 0
else:
    print("sports_game_df 로드 실패 또는 대상 지역 내 경기장 없음.")
    base_df['game_count'] = 0
    base_df['sports_types'] = '없음'
    base_df['is_post_season'] = 0

base_df['game_count'] = base_df['game_count'].fillna(0).astype(int)
base_df['sports_types'] = base_df['sports_types'].fillna('없음')
base_df['is_post_season'] = base_df['is_post_season'].fillna(0).astype(int)

# 5. 교통 사고 데이터 가져오기
if not traffic_accident_df.empty:
    accidents_in_region_df = traffic_accident_df[traffic_accident_df['region'] == TARGET_REGION]
    accidents_in_region_df = accidents_in_region_df.rename(columns={'accident_date': 'date'})
    if not accidents_in_region_df.empty:
        accident_summary_df = accidents_in_region_df.groupby('date').agg(
            accident_count_sum=('accident_count', 'sum')
        ).reset_index()
        accident_summary_df = accident_summary_df.rename(columns={'accident_count_sum': 'accident_count'})
        
        base_df = pd.merge(base_df, accident_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 교통사고 정보 없음.")
        base_df['accident_count'] = 0
else:
    print("traffic_accident_df 로드 실패.")
    base_df['accident_count'] = 0
    
base_df['accident_count'] = base_df['accident_count'].fillna(0).astype(int)

# 날씨 데이터 가져오기
if not weather_df.empty:
    weather_region_df = weather_df[weather_df['region'] == "수원"]
    weather_region_df = weather_region_df.rename(columns={'weather_date': 'date'})

    if not weather_region_df.empty:
        # 날씨 데이터는 하루에 여러 번 기록될 수 있으므로, 일별 집계 필요
        weather_summary_df = weather_region_df.groupby('date').agg(
            temperature=('temperature', 'mean'),
            precipitation=('precipitation', 'sum'),
            snow_depth=('snow_depth', 'sum'),
            avg_cloud_amount=('cloud_amount', 'mean') # 대표 날씨 상태 추론용
        ).reset_index()

        def get_weather_condition(row):
            if (pd.isna(row['precipitation']) or pd.isna(row['snow_depth'])) and pd.isna(row['avg_cloud_amount']):
                return '정보없음' # 데이터가 없는 경우
            if row['precipitation'] > 0 or pd.isna(row['snow_depth']) > 0:
                return '비/눈'
            elif pd.notna(row['avg_cloud_amount']):
                if row['avg_cloud_amount'] >= 7: # (0-10 기준)
                    return '흐림'
                elif row['avg_cloud_amount'] >= 3:
                    return '구름조금' # 또는 '약간흐림' 등
                else:
                    return '맑음'
            return '정보없음' # 강수량 없고 구름 정보도 없는 경우
            
        weather_summary_df['weather_condition'] = weather_summary_df.apply(get_weather_condition, axis=1)
        weather_summary_df = weather_summary_df[['date', 'temperature', 'precipitation', 'snow_depth', 'weather_condition']]
        
        base_df = pd.merge(base_df, weather_summary_df, on='date', how='left')
    else:
        print(f"{TARGET_REGION} 내 해당 기간 날씨 정보 없음.")
        base_df['temperature'] = pd.NA
        base_df['precipitation'] = pd.NA
        base_df['snow_depth'] = pd.NA
        base_df['weather_condition'] = '정보없음'
else:
    print("weather_df 로드 실패.")
    base_df['temperature'] = pd.NA
    base_df['precipitation'] = pd.NA
    base_df['snow_depth'] = pd.NA
    base_df['weather_condition'] = '정보없음'

# 공휴일 및 주말 데이터 가져오기
weekday_map_kr = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
base_df['weekday'] = base_df['date'].dt.dayofweek.map(weekday_map_kr)

try:
    import holidays
    # base_df['date']에서 연도를 뽑아와서 unique 값으로 추출
    kr_holidays = holidays.KR(years=base_df['date'].dt.year.unique().tolist()) 
    is_statutory_holiday = base_df['date'].apply(lambda d: d in kr_holidays)
    is_saturday = (base_df['weekday'] == '토')
    is_sunday = (base_df['weekday'] == '일')
    # 3. 세 가지 조건을 OR 연산자로 결합하여 is_holiday 컬럼 생성
    # (하나라도 True이면 True -> 1, 모두 False이면 False -> 0)
    base_df['is_holiday'] = (is_statutory_holiday | is_saturday | is_sunday).astype(int)
except ImportError:
    print("`holidays` 라이브러리가 설치되지 않았습니다. `pip install holidays`로 설치해주세요. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0
except Exception as e:
    print(f"공휴일 정보 처리 중 오류: {e}. 'is_holiday'는 0으로 처리됩니다.")
    base_df['is_holiday'] = 0
    
# 최종 데이터 셋 구성
final_df = base_df[[
    'date', 'region_code', 'accident_count', 'game_count', 'sports_types',
    'is_post_season', 'temperature', 'precipitation', 'snow_depth', 'weather_condition',
    'is_holiday', 'weekday'
]].copy() # SettingWithCopyWarning 방지를 위해 .copy() 사용

final_df['date'] = final_df['date'].dt.strftime('%Y-%m-%d')
final_df['temperature'] = pd.to_numeric(final_df['temperature'], errors='coerce').round(1)
final_df['precipitation'] = pd.to_numeric(final_df['precipitation'], errors='coerce').round(1)



울산 남구 내 경기장 코드: ['US01', 'US02', 'US03']


# 4. 최종 데이터 셋 csv 파일로 저장 및 DataFrame 출력 

In [ ]:
from datetime import datetime

final_df.to_csv(f"./results/2nd-dataset_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv", index=False)
print("\n--- Final Dataset ---")
final_df


--- Final Dataset ---


,date,region_code,accident_count,game_count,sports_types,is_post_season,temperature,precipitation,snow_depth,weather_condition,is_holiday,weekday
0,2023-01-01,울산 남구,0,0,없음,0,-2.0,0.0,0.0,맑음,1,일
1,2023-01-02,울산 남구,5,0,없음,0,-5.2,0.0,0.0,맑음,0,월
2,2023-01-03,울산 남구,3,0,없음,0,-6.2,0.0,0.0,맑음,0,화
3,2023-01-04,울산 남구,2,0,없음,0,-3.5,0.0,0.0,맑음,0,수
4,2023-01-05,울산 남구,2,0,없음,0,-3.6,0.0,0.0,구름조금,0,목
...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024-12-27,울산 남구,1,0,없음,0,-2.7,0.0,0.6,맑음,0,금
727,2024-12-28,울산 남구,1,0,없음,0,-3.1,0.0,0.9,맑음,1,토
728,2024-12-29,울산 남구,0,0,없음,0,0.5,0.0,0.0,구름조금,1,일
729,2024-12-30,울산 남구,6,0,없음,0,5.0,0.0,0.0,구름조금,0,월
